In [1]:
from PIL import Image # для работы с изображениями
import numpy as np # для массивов пикселей
from collections import deque # для реализация BFS (поиска в ширину)

In [2]:
def load_image(path):
    img = Image.open(path).convert('L')  # переводим всё изображение в оттенки серого (работаем с яркостью, а не с цветами)
    return np.array(img) # получаем наше изображение в виде массива пикселей

In [12]:
def threshold_image(gray, thresh=200): # задаём порог для яркости в 128
    return (gray > thresh).astype(np.uint8) # делим всё изображение на тёмные и светлые пиксели, сравнивая с заданным порогом

In [4]:
def find_connected_components(binary):
    h, w = binary.shape # высота и ширина изображения
    visited = np.zeros_like(binary, dtype=bool) # здесь храним просмотренные пиксели
    components = [] # здесь храним объекты

    def bfs(x, y): # используем метод обхода в ширину для выделения объектов на изображении
        queue = deque([(x, y)])
        component = []
        visited[x, y] = True
        while queue:
            cx, cy = queue.popleft()
            component.append((cx, cy))
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    nx, ny = cx + dx, cy + dy
                    if 0 <= nx < h and 0 <= ny < w and not visited[nx, ny] and binary[nx, ny]:
                        visited[nx, ny] = True
                        queue.append((nx, ny))
        return component

    for i in range(h): # обход всех пикселей
        for j in range(w):
            if binary[i, j] and not visited[i, j]:
                comp = bfs(i, j)
                components.append(comp)
    return components

In [5]:
def select_most_prominent(components, gray): # выбираем самый выделяющийся из найденных объектов (самый большой и яркий)
    def prominence(comp):
        values = [gray[x, y] for x, y in comp]
        return len(comp) * (np.mean(values) - np.mean(gray))  # area × contrast
    return max(components, key=prominence)

In [6]:
def isolate_component(gray, component): # превращаем все, кроме выбранного объекта, в чёрные точки
    mask = np.zeros_like(gray)
    for x, y in component:
        mask[x, y] = gray[x, y]
    return mask

In [7]:
def save_image(array, path): # преобразуем массив обратно в изображение и сохраняем
    img = Image.fromarray(array.astype(np.uint8))
    img.save(path)

In [17]:
gray = load_image('./additional/planets.jpg')
binary = threshold_image(gray, thresh=128)
components = find_connected_components(binary)
main_object = select_most_prominent(components, gray)
result = isolate_component(gray, main_object)
save_image(result, './additional/planets_output.jpg')